<a href="https://colab.research.google.com/github/Yassmina-Abdo/realtime-head-pose-estimation/blob/main/Notebooks/models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Liberaries

In [78]:
import numpy as np
import pickle
import pandas as pd
from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import LinearSVR, SVR,SVC
from sklearn.preprocessing import MinMaxScaler,StandardScaler
from sklearn.metrics import mean_squared_error,mean_absolute_error, r2_score
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure

## Reading the prepared dataset and split it

In [79]:
path_local='../Data/media_with_angles.csv'
path_drive='/content/drive/MyDrive/Head Pose Estimation ML Project/Dataset/Data/data468_normed_low.csv'
data =pd.read_csv(path_drive)
data.head()

,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,...,y463,y464,y465,y466,y467,y468,yaw,pitch,roll,image_name
0,0.0,0.768221,0.042679,0.298753,0.896258,0.768221,0.256074,-2.987527,0.256074,0.384111,...,-22.0,-86.0,-83.0,-80.0,-91.0,-95.0,0.725284,-0.140406,-0.174674,image03195
1,0.0,-0.119313,-0.053028,-0.198854,-0.132570,-0.132570,-0.092799,-0.755647,-0.106056,-0.119313,...,-15.0,-74.0,-72.0,-70.0,-84.0,-86.0,-0.233745,-0.116700,0.095809,image03247
2,0.0,-0.257694,0.000000,-0.227377,-0.303170,-0.257694,-0.090951,-0.212219,-0.090951,-0.151585,...,-23.0,-76.0,-72.0,-70.0,-86.0,-92.0,-1.330352,0.003072,0.362599,image03214
3,0.0,-0.011573,0.023147,-0.081013,0.000000,0.000000,0.046293,-0.763836,0.057866,0.057866,...,-21.0,-105.0,-99.0,-94.0,-121.0,-125.0,-0.040907,-0.474087,0.019801,image03238
4,0.0,0.126856,0.063428,0.047571,0.158570,0.174427,0.206142,-0.856280,0.237856,0.269570,...,-24.0,-94.0,-91.0,-88.0,-99.0,-102.0,0.205732,-0.241359,-0.150232,image03236


In [80]:
X = data.iloc[:,:-4]
X.head(1)

,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,...,y459,y460,y461,y462,y463,y464,y465,y466,y467,y468
0,0.0,0.768221,0.042679,0.298753,0.896258,0.768221,0.256074,-2.987527,0.256074,0.384111,...,-24.0,-27.0,-24.0,-23.0,-22.0,-86.0,-83.0,-80.0,-91.0,-95.0


In [81]:
y = data.iloc[:,-4:-1]
y.head(1)

,yaw,pitch,roll
0,0.725284,-0.140406,-0.174674


## Scaling

In [6]:
scaler = MinMaxScaler()
x_scaled = scaler.fit_transform(X.to_numpy())

In [7]:
pca = PCA(n_components=200)
X_pca = pca.fit_transform(x_scaled)

In [104]:
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42, shuffle=True)

## Helping methods

In [55]:
def performance_metrics(y_true, y_pred):
  print(f"mean square error = {mean_squared_error(y_true, y_pred)}")
  print(f"mean absolute error = {mean_absolute_error(y_true, y_pred)}")
  print(f"R2-score = {r2_score(y_true, y_pred)}")

## **Model 1: RandomForest**




In [ ]:
clf = MultiOutputRegressor(RandomForestRegressor(max_depth=6, random_state=42, bootstrap=True),n_jobs=-1)
clf.fit(x_train, y_train)

In [106]:
print(f"training score: {clf.score(x_train, y_train, sample_weight=None)}")
print(f"testing score: {clf.score(x_test, y_test, sample_weight=None)}")

training score: 0.8793670627720029
testing score: 0.7148822620496736


In [107]:
# save model
with open('/content/drive/MyDrive/Head Pose Estimation ML Project/Saved Models/model_Rm.pkl', 'wb') as file:
        pickle.dump(clf, file)

## **Model 2: SVR**
  

In [83]:
# define base model
model = SVR(kernel='rbf', C=300, degree=4,coef0=1,max_iter=4)  #kernel='poly', C=100, gamma='auto', degree=3, epsilon=.1,coef0=1
#model = SVR(kernel='poly', C=100, gamma='auto', degree=3, epsilon=.1,coef0=1)  #kernel='poly', C=100, gamma='auto', degree=3, epsilon=.1,coef0=1

# define the direct multioutput wrapper model
wrapper = MultiOutputRegressor(model)
wrapper.fit(x_train, y_train)
y_pred = wrapper.predict(x_test)

In [84]:
print(f"training score: {wrapper.score(x_train, y_train, sample_weight=None)}")
print(f"testing score: {wrapper.score(x_test, y_test, sample_weight=None)}")

training score: 0.7328009263654853
testing score: 0.7317789035299267


In [85]:
# save model
with open('/content/drive/MyDrive/Head Pose Estimation ML Project/Saved Models/model_svr.pkl', 'wb') as file:
        pickle.dump(wrapper, file)

## **Model 3: GradientBoost**



In [69]:
from sklearn.ensemble import GradientBoostingRegressor
gbr = GradientBoostingRegressor(random_state=42,n_estimators=20,learning_rate=0.05,max_depth=6) #random_state=42,n_estimators=100,learning_rate=0.05
model = MultiOutputRegressor(estimator=gbr,n_jobs=-1)

In [70]:
model.fit(x_train, y_train)
ypred = model.predict(x_test)

In [71]:
print(f"training score: {model.score(x_train, y_train)}")
print(f"testing score: {model.score(x_test, y_test)}")

training score: 0.7894682660089803
testing score: 0.6205588361840744


In [ ]:
# save model
with open('/content/drive/MyDrive/Head Pose Estimation ML Project/Saved Models/model_GB.pkl', 'wb') as file:
        pickle.dump(model, file)

## **Model 3: XGBoost**



In [90]:
from xgboost.sklearn import XGBRegressor
xbg = XGBRegressor(random_state=42, max_depth=5, n_estimators=300,reg_alpha=0.1)
model = MultiOutputRegressor(estimator=xbg,n_jobs=-1)

In [91]:
model.fit(x_train, y_train)
ypred = model.predict(x_test)

In [92]:
print(f"training score: {model.score(x_train, y_train)}")
print(f"testing score: {model.score(x_test, y_test)}")

training score: 0.9993191456335947
testing score: 0.6962723178396897


In [93]:
# save model
with open('/content/drive/MyDrive/Head Pose Estimation ML Project/Saved Models/model_XGB.pkl', 'wb') as file:
        pickle.dump(model, file)

In [ ]:
w